In [ ]:
# mount google colab

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages

!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_heading_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_fulltext.csv')
left_heading_dataset = left_heading_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
left_heading_dataset = left_heading_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text_cleaned"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_left_heading_dataset = left_heading_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_heading_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2634 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2228 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1974 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1492 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1193 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1027 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3069 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1619 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_left_heading_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
#finetune the pretrained model on the dataset
training_args = TrainingArguments(
    output_dir="distillgpt2_left_fulltext_cleaned_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 25.49


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/312M [00:00<?, ?B/s]

Upload file runs/Jul25_12-38-57_62cbeb27e7d4/events.out.tfevents.1690292853.62cbeb27e7d4.4722.1:   0%|        …

Upload file runs/Jul25_12-38-57_62cbeb27e7d4/events.out.tfevents.1690288747.62cbeb27e7d4.4722.0:   0%|        …

To https://huggingface.co/tobijen/distillgpt2_left_fulltext_cleaned_torch
   5b784df..a685d52  main -> main

   5b784df..a685d52  main -> main

To https://huggingface.co/tobijen/distillgpt2_left_fulltext_cleaned_torch
   a685d52..2c16dc1  main -> main

   a685d52..2c16dc1  main -> main



'https://huggingface.co/tobijen/distillgpt2_left_fulltext_cleaned_torch/commit/a685d52449a14543259a02ae7131630933e8f51d'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/distillgpt2_left_fulltext_cleaned_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/distillgpt2_left_fulltext_cleaned_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Gay marriage is supposed to remain legal, said the law, which seeks to protect gay marriage, and in recent years has been largely successful.AdvertisementThe Supreme Court, in its recent ruling, struck down same-sex marriage in Michigan, saying it was a violation of the federal law’s constitutional right to marry. The court upheld that law in 2015.A spokeswoman for Arkansas Gov. Asa Hutchinson said that the court will hear oral arguments soon.AdvertisementThe Department of Defense's spokesman, Col. John Kirby"}]
social_2  =>  Abortion is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Abortion is illegal in many states.But abortion opponents have noted that such an act only allows for states to request a child with no abortion or no medical knowledge.“I cannot guarantee the right to abortion,” said Dr. Ann Marie Hennepin, a clinical and epidemiologist with the National Abortion Federation. “That law does not allow the state to restrict or otherwise deny access to abortion on any level without a guarantee of access” for all the abortion providers.In a blog post'}]
education_1  =>  Public education is


...

[{'generated_text': 'Charter schools are also doing a good job explaining how they are creating jobs. They’re also working on learning strategies to promote the middle class and working families. The administration and local governments are doing good, but they’re at the end of their day in office.And they’re doing good.”The president’s remarks aren’t a bad sign for his party. They suggest he’s already made a bad impression over the past six months by failing to engage'}]
economic_1  =>  The increase of taxes is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The increase of taxes is far from the only thing preventing a rise in the personal wealth tax rate. On one level, the rise in the estate tax has helped keep households happy while allowing wealthier Americans to pay the tax on the income of the wealthy. It has resulted in many tax cuts for the wealthy while also lowering rates on the top 1% of Americans.But on another, it is still far from clear that such tax cuts make everyone less financially connected, which means that any change in wealth could take years.The U'}]
economic_2  =>  Government regulations on businesses are


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Government regulations on businesses are very limited and so no federal regulation will be in place on how much information the industry has about its business.”The law goes into effect Aug. 1, after a handful of states that have legalized assisted same-sex marriage have passed bills allowing businesses to refuse to allow same-sex couples to be wed.The measure requires states to allow religious organizations to establish religious unions and bans the use of non-discrimination policies, such as religious or ethnic identity or religious expression. The bill allows companies to'}]


In [ ]:
generated_text_dict

{'social_1': "Gay marriage is supposed to remain legal, said the law, which seeks to protect gay marriage, and in recent years has been largely successful.AdvertisementThe Supreme Court, in its recent ruling, struck down same-sex marriage in Michigan, saying it was a violation of the federal law’s constitutional right to marry. The court upheld that law in 2015.A spokeswoman for Arkansas Gov. Asa Hutchinson said that the court will hear oral arguments soon.AdvertisementThe Department of Defense's spokesman, Col. John Kirby",
 'social_2': 'Abortion is illegal in many states.But abortion opponents have noted that such an act only allows for states to request a child with no abortion or no medical knowledge.“I cannot guarantee the right to abortion,” said Dr. Ann Marie Hennepin, a clinical and epidemiologist with the National Abortion Federation. “That law does not allow the state to restrict or otherwise deny access to abortion on any level without a guarantee of access” for all the abor

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "distilgpt2_left_fulltext_cleaned": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)